In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from openpyxl import load_workbook
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams.update({'font.size': 22})

# Loading the data


In [3]:
with open("OneDrive_1_05-10-2018/Headers.csv", 'rt') as f:
    headers = pd.read_csv(f)

In [6]:
data = dict()
for file in range(2013,2019):
    print(file)
    with open("OneDrive_1_05-10-2018/%s.csv"%file, 'rt') as f:
        data[file] = pd.read_csv(f,header=None)
        data[file].columns = headers.columns
        data[file]["year"] = file

2013
2014
2015
2016
2017
2018


In [7]:
all_data = pd.concat([item for _,item in data.items()])


In [8]:
for col in ['userid','company','company_de']:
    all_data[col] = pd.to_numeric(all_data[col],downcast='unsigned')

/Users/severinhatt/Virtualenvs/thinkful/base_plotting/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [9]:
wb = load_workbook(filename='OneDrive_1_05-10-2018/SRM Trades.xlsx', read_only=True)
ws = wb.active
srm = list()
for row in ws.rows:
    srm_list = {'id': row[0].value ,'trade': row[1].value , 'active': row[2].value}
    srm.append(srm_list)
srm_df = pd.DataFrame.from_records(srm[1:]) #first line is the header

In [ ]:
all_data=all_data.merge(srm_df,left_on='trade',right_on="id").drop("trade_x",axis=1).rename({"trade_y":"trade"},axis=1)

In [ ]:
all_data[all_data.year == 2013].sample(25)

In [ ]:
#In 2013 SwipeIn seems to only by a time with no dates...
all_data_but_2013 =  all_data[all_data.year != 2013].copy()

In [ ]:
for col in ['swipeIn','swipeOut']:
    all_data_but_2013[col] = pd.to_datetime(all_data_but_2013[col])

In [ ]:
all_data_but_2013["work_hours"] = (all_data_but_2013.swipeOut - all_data_but_2013.swipeIn).dt.seconds/3600

In [ ]:
all_data_but_2013.info()

In [ ]:
all_data_but_2013.describe()

In [ ]:
all_data_but_2013["swipeIn_date"] =all_data_but_2013.swipeIn.dt.date
all_data_but_2013["swipeIn_date"] = pd.to_datetime(all_data_but_2013["swipeIn_date"])

In [ ]:
swipe_per_day_per_user = all_data_but_2013[~all_data_but_2013.work_hours.isna()].groupby(["trade","userid","swipeIn_date"]).agg({'userid': 'count', 'work_hours': ['sum','max',"median",'min']})

In [ ]:
swipe_per_day_per_user.columns = swipe_per_day_per_user.columns.droplevel()

In [ ]:
swipe_per_day_per_user.head()

In [ ]:
people_per_day=swipe_per_day_per_user.reset_index().groupby(["swipeIn_date"])\
.agg({'userid': 'count', 'sum': 'sum','max':'max',"median":"median",'min':'min'})\
.rename(columns={'userid':'user_count',"sum":"sum_hours"})\
.reset_index()

In [ ]:
people_per_day["weekday"]= people_per_day.swipeIn_date.dt.weekday

In [ ]:
people_per_day[people_per_day.user_count==np.max(people_per_day.user_count)]

In [ ]:
all_data_but_2013[all_data_but_2013.swipeIn_date=='2016-06-21']

In [ ]:
sns.boxenplot(data=people_per_day,x="weekday",y="user_count")


In [ ]:
sns.set_style("whitegrid")
sns.boxenplot(data=people_per_day,x="weekday",y="sum_hours")\
.set_title("Distribution of the number of hours worked per day depending on the day of the week (0=> Monday, 6=> Sunday)");


In [ ]:
trade_per_day=swipe_per_day_per_user.reset_index().groupby(["trade","swipeIn_date"])\
.agg({'trade': 'count', 'sum': 'sum','max':'max',"median":"median",'min':'min'})\
.rename(columns={'trade':'trade_count',"sum":"sum_hours"})\
.reset_index()

In [ ]:
#We can graphically confirm that people indeed work 8 hours per day!
sns.scatterplot(data=trade_per_day[trade_per_day.swipeIn_date=='2016-06-21'],
                x="trade_count",y="sum_hours",hue="trade",
               legend=False)\
.set_title("Number of people working and hours worked per trade category for 21 June 2016");

In [ ]:
all_data_but_2013["swipe_in_out_same_day"] = all_data_but_2013.swipeIn.dt.date == all_data_but_2013.swipeOut.dt.date

In [ ]:
#Only a very small portion of the swipes have different date for swipe in and out.
sum(all_data_but_2013["swipe_in_out_same_day"])/all_data_but_2013.shape[0]